In [1]:
import re
import random
import sys
from math import ceil,floor,fmod,gcd,sqrt,inf
from bisect import bisect_left
from collections import defaultdict,Counter,deque,OrderedDict
from functools import cache, reduce, cmp_to_key
from itertools import accumulate, combinations, permutations
from heapq import nsmallest, nlargest, heappushpop, heapify, heappop, heappush
from copy import deepcopy
from typing import *
from string import ascii_lowercase, ascii_uppercase

if "PyPy" in sys.version:
    import pypyjit; pypyjit.set_param('max_unroll_recursion=-1')

limits = [100000, 10000, 5000, 2000]
for limit in limits:
    try:
        sys.setrecursionlimit(limit)
        break
    except:
        continue 

def fstream(func):
    def wrapper():
        input_file = open('data.in', 'r')
        sys.stdin = input_file
        output_file = open('data.out', 'w')
        sys.stdout = output_file
        func()
        input_file.close()
        sys.stdin = sys.__stdin__
        output_file.close()
        sys.stdout = sys.__stdout__
    return wrapper

input = lambda: sys.stdin.readline().rstrip("\r\n")

def I():
    return input()
 
def II():
    return int(input())
 
def MII():
    return map(int, input().split())
 
def LI():
    return list(input().split())
 
def LII():
    return list(map(int, input().split()))

def for_t(func):
    def wrapper():
        T=II()
        for _ in range(T):
            func()
    return wrapper

## 测试去重的速度

In [36]:
import timeit

# 定义测试函数
def test_set(a):
    x = list(set(a))
    x.sort()

def test_list_comprehension(a):
    a.sort()
    x = [a[i] for i in range(len(a)) if a[i] != a[i - 1]]

# 随机化生成数组
random.seed(41)  # 设置随机种子以确保结果可重现
a = [random.randint(1, 10000000) for _ in range(10000)]

# 测试 set 方法的耗时
set_time = timeit.timeit(lambda: test_set(a), number=1000)

# 测试列表推导式的耗时
list_comp_time = timeit.timeit(lambda: test_list_comprehension(a), number=1000)

# 输出结果
print("set 方法耗时:", set_time)
print("列表推导式耗时:", list_comp_time)

set 方法耗时: 2.552644799929112
列表推导式耗时: 1.5086278999224305


In [1]:
"""
AVL 平衡二叉树
通过四种旋转来维护平衡
写起来长了点,理解难度还好
"""
import sys
if "PyPy" in sys.version:
    import pypyjit; pypyjit.set_param('max_unroll_recursion=-1')


class AVLNode:
    __slots__ = ('key', 'left', 'right', 'height')
    def __init__(self, key, left=None, right=None, height=0):
        self.key = key
        self.left = left
        self.right = right
        self.height = height

    def __str__(self):
        return 'AVLNode(%s)' % self.key

    def __repr__(self):
        return 'AVLNode(%s)' % self.key

class AVLTree:

    __slots__ = 'root'
    def __init__(self):
        self.root = None
    def get_height(self, node):
        if not node: return 0
        return node.height
    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)
    def update_height(self, node):
        if not node:
            return
        node.height = 1 + max(self.get_height(node.left), self.get_height(node.right))

    def left_rotate(self, node):
        """
        左旋: 左右等于左(左子树的右节点=左节点),父子交换
        """
        l=node.left
        node.left=l.right
        l.right=node
        self.update_height(node)
        self.update_height(l)
        return l
    def right_rotate(self, node):
        """
        右旋: 右左等于右(右子树的左节点=右节点),父子交换
        """
        r=node.right
        node.right=r.left
        r.left=node
        self.update_height(node)
        self.update_height(r)
        return r
    
    def rebalance(self, node):

        balance = self.get_balance(node)

        # 左-左情况，进行右旋转
        if balance > 1 and self.get_balance(node.left) >= 0:
            node=self.right_rotate(node)

        # 左-右情况，先进行左旋转，再进行右旋转
        elif balance > 1:
            node.left = self.left_rotate(node.left)
            node=self.right_rotate(node)

        # 右-右情况，进行左旋转
        elif balance < -1 and self.get_balance(node.right) <= 0:
            node=self.left_rotate(node)

        # 右-左情况，先进行右旋转，再进行左旋转
        elif balance < -1:
            node.right = self.right_rotate(node.right)
            node=self.left_rotate(node)

        return node
    
    def _BST_insert(self,node,x):
        if not node:
            return AVLNode(x)
        if x<node.key:
            node.left=self._BST_insert(node.left,x)
        elif x>node.key:
            node.right=self._BST_insert(node.right,x)
        return node

    def _insert(self, node,x):
        node=self._BST_insert(node,x)
        if not node: return None
        # self.update_height(node)
        # node=self.rebalance(node)
        return node
    
    def insert(self, x):
        self.root=self._insert(self.root,x)
    
    def find_min(self, node):
        current = node
        while current.left:
            current = current.left
        return current
    
    def _BST_delete(self,node,x):
        if not node:
            return node
        if x<node.key:
            node.left=self._BST_delete(node.left,x)
        elif x>node.key:
            node.right=self._BST_delete(node.right,x)
        else:
            if not node.left:
                return node.right
            elif not node.right:
                return node.left
            else:
                min_node = self.find_min(node.right)
                node.key = min_node.key
                node.right = self._BST_delete(node.right, min_node.key)
        return node
    
    def _delete(self, node, key):
        node=self._BST_delete(node,key)
        if not node: return None
        self.update_height(node)
        node = self.rebalance(node)
        return node

    def delete(self,key):
        self.root = self._delete(self.root, key)

    def get_size(self, node):
        if not node:
            return 0
        return 1 + self.get_size(node.left) + self.get_size(node.right)
    
    def get_rank(self, node, x):
        if not node:
            return 0

        if x < node.key:
            return self.get_rank(node.left, x)
        elif x > node.key:
            return 1 + self.get_size(node.left) + self.get_rank(node.right, x)
        else:
            return self.get_size(node.left) + 1

    def rank(self, x):
        return self.get_rank(self.root, x)    
    
    def get_kth_largest(self, node, k):
        if not node:
            return None

        size_right = self.get_size(node.right)
        if k <= size_right:
            return self.get_kth_largest(node.right, k)
        elif k == size_right + 1:
            return node.key
        else:
            return self.get_kth_largest(node.left, k - size_right - 1)

    def kth_largest(self, k):
        return self.get_kth_largest(self.root, k)
    
    def get_predecessor(self, node, x):
        if not node:
            return None

        if x <= node.key:
            return self.get_predecessor(node.left, x)
        else:
            predecessor = self.get_predecessor(node.right, x)
            if predecessor is None:
                return node.key
            else:
                return predecessor

    def predecessor(self, x):
        return self.get_predecessor(self.root, x)
    
    def get_successor(self, node, x):
        if not node:
            return None

        if x >= node.key:
            return self.get_successor(node.right, x)
        else:
            successor = self.get_successor(node.left, x)
            if successor is None:
                return node.key
            else:
                return successor

    def successor(self, x):
        return self.get_successor(self.root, x)
    
    def print_tree(self):
        self.print_inorder(self.root)

    def print_inorder(self, node):
        if node:
            self.print_inorder(node.left)
            print(node, end=" ")
            self.print_inorder(node.right)
    

    

In [2]:
avl = AVLTree()

In [7]:
avl.print_tree()

AVLNode(1) 

In [6]:
avl.insert(1)